In [ ]:
%%configure -f
{"conf": 
 { 
     "spark.pyspark.virtualenv.enabled": "false",
     "spark.yarn.appMasterEnv.YARN_CONTAINER_RUNTIME_TYPE":"docker",
     "spark.executorEnv.YARN_CONTAINER_RUNTIME_DOCKER_CLIENT_CONFIG":"hdfs:///user/hadoop/config.json",
     "spark.executorEnv.YARN_CONTAINER_RUNTIME_DOCKER_IMAGE":"574404580792.dkr.ecr.us-east-1.amazonaws.com/scalableml:s3spark",
     "spark.executor.instances":"2",
     "spark.yarn.appMasterEnv.YARN_CONTAINER_RUNTIME_DOCKER_CLIENT_CONFIG":"hdfs:///user/hadoop/config.json",
     "spark.yarn.appMasterEnv.YARN_CONTAINER_RUNTIME_DOCKER_IMAGE":"574404580792.dkr.ecr.us-east-1.amazonaws.com/scalableml:s3spark"
 }
}

In [1]:
import numpy as np
print(np.__version__)

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1623599779960_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1.16.5

In [2]:
import os
import boto3
import pandas as pd
import sys

if sys.version_info[0] < 3: 
    from StringIO import StringIO # Python 2.x
else:
    from io import StringIO # Python 3.x

# get your credentials from environment variables
aws_id = '****************'
aws_secret = '*******************'

client = boto3.client('s3', aws_access_key_id=aws_id,
                        aws_secret_access_key=aws_secret)

csv_obj = client.get_object(Bucket='spark-anand', Key='forecasted_1.csv')
body = csv_obj['Body']
csv_string = body.read().decode('utf-8')

df = pd.read_csv(StringIO(csv_string))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Predict Humidity for Multiple Weather Parameters (Time Series Data)

The daily temperate, wind, rainfall and humidity of a location is recorded and noted in the input dataset from 1990~2020s. 

Build a time series model to predict the humidity in Y2021, given these features: temperate, wind, rainfall. 2020Q4 data can be used as test dataset, while the remaining for training the model. To verify the model, you can use any metric to compare prediction and actual humidity values in 2020 Q4.


In [3]:
import sys
import math
import boto3
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.4f' % x)
import seaborn as sns
sns.set_context("paper", font_scale=1.3)
sns.set_style('white')
import warnings
warnings.filterwarnings('ignore')
from time import time
import matplotlib.ticker as tkr
from scipy import stats
from statsmodels.tsa.stattools import pacf

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

# Importing the statistics module
from statistics import mean
from statistics import median

# To load the input data
import pandas as pd
import numpy as np
from datetime import datetime
from matplotlib import pyplot


if sys.version_info[0] < 3: 
    from StringIO import StringIO # Python 2.x
else:
    from io import StringIO # Python 3.x

# import tensorflow as tf 
# tf.test.gpu_device_name() 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


# Loading  Data

In [4]:

######################################################
# get your credentials from environment variables
aws_id = '***************'
aws_secret = '*********************'

client = boto3.client('s3', aws_access_key_id=aws_id,
                        aws_secret_access_key=aws_secret)


def openFile(fileName):
        csv_obj = client.get_object(Bucket='spark-anand', Key=fileName)
        body = csv_obj['Body']
        csv_string = body.read().decode('utf-8')

        df = pd.read_csv(StringIO(csv_string))
        return df
######################################################
                                    
# Read the s&p 500 input data set and sorting based on date.
observed = openFile("observed_1.csv")
observed.head()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

       date    temp    wind  rainfall  humidity
0  19900101 28.4000 17.9600   20.4000   32.1800
1  19900102 35.5000  2.2300    0.0000   22.8400
2  19900103 17.4000  9.0600    0.0000   29.3800
3  19900104 28.4000  1.5700    0.0000   26.3000
4  19900105 28.3000  0.3000    0.0000   26.7500

# Data Transformations

In [5]:
observed_filtered = observed[observed['temp'] < 100] 
len(observed_filtered)

from datetime import datetime
day_of_year = datetime.now().timetuple().tm_yday
day_of_year


# datetime.now()
date_time_obj = datetime.strptime("040205", '%y%m%d')
date_time_obj.timetuple().tm_yday


def convert_to_date(date):
    date_as_string = str(date)
    date_as_string = date_as_string[2:]
    date_time_obj = datetime.strptime(date_as_string, '%y%m%d')
    return date_time_obj
    
def convert_to_daynum(date):
    date_as_string = str(date)
    date_as_string = date_as_string[2:]
    date_time_obj = datetime.strptime(date_as_string, '%y%m%d')

    return date_time_obj.timetuple().tm_yday

#     return int(year_as_string)

observed_filtered['day'] = observed_filtered['date'].apply(convert_to_daynum)
observed_filtered['date'] = observed_filtered['date'].apply(convert_to_date)

observed_filtered.head()


df = observed_filtered

# df['date']=pd.to_datetime(df['date'])
df['year'] = df['date'].apply(lambda x: x.year)
df['week'] = df['date'].apply(lambda x: x.week)
df['month'] = df['date'].apply(lambda x: x.month)
df['dayinmonth'] = df['date'].apply(lambda x: x.day)
df['humScale'] = df['temp'] * df['rainfall'] 

df.sort_values('date', inplace=True, ascending=True)
# df=df.loc[:,['date','year','quarter','month','dayinmonth', 'temp', 'wind', 'rainfall', 'day', 'humidity']]
df=df.loc[:,['week', 'temp', 'wind', 'rainfall', 'day', 'humScale', 'humidity']]
df = df.reset_index(drop=True)
df.head(10)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   week    temp    wind  rainfall  day  humScale  humidity
0     1 28.4000 17.9600   20.4000    1  579.3600   32.1800
1     1 35.5000  2.2300    0.0000    2    0.0000   22.8400
2     1 17.4000  9.0600    0.0000    3    0.0000   29.3800
3     1 28.4000  1.5700    0.0000    4    0.0000   26.3000
4     1 28.3000  0.3000    0.0000    5    0.0000   26.7500
5     1 35.4000 14.0400    0.0000    6    0.0000   20.7900
6     1 11.4000  5.6700    0.0000    7    0.0000   32.9600
7     2 29.5000 18.3600    0.0000    8    0.0000   21.8500
8     2 12.8000 19.2400    0.0000    9    0.0000   29.5500
9     2 32.2000  9.2700    0.0000   10    0.0000   22.7400

In [6]:

# Visualization Module - Humidity

humidity = df['humidity']
x = np.arange(0, len(humidity))
print(len(humidity))

plt.figure(figsize=(15,5))
plt.plot(x, humidity, label="Humidity")

plt.legend()
plt.title("Humidity vs Time", fontsize=20)
plt.ylabel("Humidity", fontsize=15)
plt.xlabel("Time", fontsize=15)
plt.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

11127

In [7]:

import pyspark
from time import time
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession


if not 'sc' in globals():
    sc = SparkContext('local')
    spark = SparkSession(sc)

from pyspark.ml.feature import VectorAssembler


# Assemble all the features with VectorAssembler
def assembleFeatures(dfRDD):

    required_features = ['week',
                        'temp',
                        'wind',
                        'rainfall',
                        'day',
                        'humScale'
                       ]

    assembler = VectorAssembler(inputCols=required_features, outputCol='features')
    transformed_data = assembler.transform(dfRDD)
    
    return transformed_data



def trainRFmodel(trainingData):

    # Train a RandomForest model.
    rf = RandomForestRegressor(featuresCol="features")

    # Chain indexer and forest in a Pipeline
    pipeline = Pipeline(stages=[rf])

    # Train model.  This also runs the indexer.
    model = pipeline.fit(trainingData)
    
    return model



def testRFmodel(model, testData, isLabelled):

    # Make predictions.
    predictions = model.transform(testData)

    # Select example rows to display.
    if isLabelled:
        predictions.select("prediction", "label", "features").show(5)
        
        # Select (prediction, true label) and compute test error
        evaluator = RegressionEvaluator(
            labelCol="label", predictionCol="prediction", metricName="rmse")
        rmse = evaluator.evaluate(predictions)
        print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
    else:
        predictions.select("prediction", "features").show(5)

    rfModel = model.stages[0]
    print(rfModel)  # summary only
    
    return predictions.select("prediction", "features")



df.columns = ['week', 'temp', 'wind', 'rainfall', 'day', 'humScale', 'label']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:

dfRDD = spark.createDataFrame(df)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
dfRDD.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(week=1, temp=28.4, wind=17.96, rainfall=20.4, day=1, humScale=579.3599999999999, label=32.18)

In [10]:
dfRDD.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----+-----+--------+---+------------------+-----+
|week|temp| wind|rainfall|day|          humScale|label|
+----+----+-----+--------+---+------------------+-----+
|   1|28.4|17.96|    20.4|  1| 579.3599999999999|32.18|
|   1|35.5| 2.23|     0.0|  2|               0.0|22.84|
|   1|17.4| 9.06|     0.0|  3|               0.0|29.38|
|   1|28.4| 1.57|     0.0|  4|               0.0| 26.3|
|   1|28.3|  0.3|     0.0|  5|               0.0|26.75|
|   1|35.4|14.04|     0.0|  6|               0.0|20.79|
|   1|11.4| 5.67|     0.0|  7|               0.0|32.96|
|   2|29.5|18.36|     0.0|  8|               0.0|21.85|
|   2|12.8|19.24|     0.0|  9|               0.0|29.55|
|   2|32.2| 9.27|     0.0| 10|               0.0|22.74|
|   2|27.9|16.92|     0.0| 11|               0.0|23.38|
|   2|13.8|10.96|    27.7| 12|            382.26|44.16|
|   2|25.3|13.89|    57.6| 13|           1457.28|52.14|
|   2|39.8|19.28|    52.4| 14|           2085.52|42.59|
|   3|28.1| 4.26|    26.9| 15|            755.89

In [ ]:
transformed_data = assembleFeatures(dfRDD)

In [11]:
# Simple reduceByKey example in python
# creating PairRDD x with key value pairs
from time import time

start = time()

x = sc.parallelize([("a", 1), ("b", 1), ("a", 1), ("a", 1),
                    ("b", 1), ("b", 1), ("b", 1), ("b", 1)]*1000000, 8)
 
# Applying reduceByKey operation on x
y = x.reduceByKey(lambda accum, n: accum + n)
print(y.collect())

print(f'Time taken to run: {time() - start} seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('a', 3000000), ('b', 5000000)]
Time taken to run: 3.694195032119751 seconds

In [2]:
dfRDD = spark.createDataFrame(x)
dfRDD.head(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(_1='a', _2=1), Row(_1='b', _2=1), Row(_1='a', _2=1), Row(_1='a', _2=1), Row(_1='b', _2=1)]

In [ ]:



#df.columns = ['week', 'temp', 'wind', 'rainfall', 'day', 'humScale', 'label']

#dfRDD = spark.createDataFrame(df)

transformed_data = assembleFeatures(dfRDD)
transformed_data.show()

# Split the data
(trainingData, testData) = transformed_data.randomSplit([0.9,0.1])

# Random Forest Regressor in Spark

In [47]:

import pyspark
from time import time
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession


if not 'sc' in globals():
    sc = SparkContext('local')
    spark = SparkSession(sc)

from pyspark.ml.feature import VectorAssembler


# Assemble all the features with VectorAssembler
def assembleFeatures(dfRDD):

    required_features = ['week',
                        'temp',
                        'wind',
                        'rainfall',
                        'day',
                        'humScale'
                       ]

    assembler = VectorAssembler(inputCols=required_features, outputCol='features')
    transformed_data = assembler.transform(dfRDD)
    
    return transformed_data



def trainRFmodel(trainingData):

    # Train a RandomForest model.
    rf = RandomForestRegressor(featuresCol="features")

    # Chain indexer and forest in a Pipeline
    pipeline = Pipeline(stages=[rf])

    # Train model.  This also runs the indexer.
    model = pipeline.fit(trainingData)
    
    return model



def testRFmodel(model, testData, isLabelled):

    # Make predictions.
    predictions = model.transform(testData)

    # Select example rows to display.
    if isLabelled:
        predictions.select("prediction", "label", "features").show(5)
        
        # Select (prediction, true label) and compute test error
        evaluator = RegressionEvaluator(
            labelCol="label", predictionCol="prediction", metricName="rmse")
        rmse = evaluator.evaluate(predictions)
        print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
    else:
        predictions.select("prediction", "features").show(5)

    rfModel = model.stages[0]
    print(rfModel)  # summary only
    
    return predictions.select("prediction", "features")



df.columns = ['week', 'temp', 'wind', 'rainfall', 'day', 'humScale', 'label']

dfRDD = spark.createDataFrame(df)

transformed_data = assembleFeatures(dfRDD)
transformed_data.show()

# Split the data
(trainingData, testData) = transformed_data.randomSplit([0.9,0.1])

start = time()
model = trainRFmodel(trainingData)
print(f'Time taken to run: {time() - start} seconds')

testRFmodel(model, testData, True)



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----+-----+--------+---+------------------+-----+--------------------+
|week|temp| wind|rainfall|day|          humScale|label|            features|
+----+----+-----+--------+---+------------------+-----+--------------------+
|   1|28.4|17.96|    20.4|  1| 579.3599999999999|32.18|[1.0,28.4,17.96,2...|
|   1|35.5| 2.23|     0.0|  2|               0.0|22.84|[1.0,35.5,2.23,0....|
|   1|17.4| 9.06|     0.0|  3|               0.0|29.38|[1.0,17.4,9.06,0....|
|   1|28.4| 1.57|     0.0|  4|               0.0| 26.3|[1.0,28.4,1.57,0....|
|   1|28.3|  0.3|     0.0|  5|               0.0|26.75|[1.0,28.3,0.3,0.0...|
|   1|35.4|14.04|     0.0|  6|               0.0|20.79|[1.0,35.4,14.04,0...|
|   1|11.4| 5.67|     0.0|  7|               0.0|32.96|[1.0,11.4,5.67,0....|
|   2|29.5|18.36|     0.0|  8|               0.0|21.85|[2.0,29.5,18.36,0...|
|   2|12.8|19.24|     0.0|  9|               0.0|29.55|[2.0,12.8,19.24,0...|
|   2|32.2| 9.27|     0.0| 10|               0.0|22.74|[2.0,32.2,9.27,0....|

# 2020 Prediction

In [ ]:

# Read the s&p 500 input data set and sorting based on date.
observed_2020 = openFile("test_input_1.csv")
observed_2020.head()


observed_2020['day'] = observed_2020['date'].apply(convert_to_daynum)
observed_2020['date'] = observed_2020['date'].apply(convert_to_date)


df = observed_2020

# df['date']=pd.to_datetime(df['date'])
df['year'] = df['date'].apply(lambda x: x.year)
df['week'] = df['date'].apply(lambda x: x.week)
df['month'] = df['date'].apply(lambda x: x.month)
df['dayinmonth'] = df['date'].apply(lambda x: x.day)

df['humScale'] = df['temp'] * df['rainfall']

df.sort_values('date', inplace=True, ascending=True)
# df=df.loc[:,['date','year','quarter','month','dayinmonth', 'temp', 'wind', 'rainfall', 'day', 'humidity']]
df=df.loc[:,['week', 'temp', 'wind', 'rainfall', 'day', 'humScale']]

df = df.reset_index(drop=True)
df.head()

dfRDD_2020 = spark.createDataFrame(df)

transformed_data_2020 = assembleFeatures(dfRDD_2020)
transformed_data_2020.show()

# Split the data
# (trainingData, testData) = transformed_data_2020.randomSplit([0.9,0.1])


predictions = testRFmodel(model, transformed_data_2020, False)
predictions.toPandas()


In [ ]:

# Read the input data set to get actual humidity values to compare with prediction
output_2020 = openFile("test_output_1.csv")
output_2020.head(10)

output_2020['humidity'].values

# len(np.array(predictions.toPandas()['prediction']))
actual_humidity = output_2020['humidity'].values
mean_squared_error(actual_humidity, np.array(predictions.toPandas()['prediction']))